In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta, timezone, date
from pysolar.solar import *
from zoneinfo import ZoneInfo
from dateutil import tz
#import datetime
#import zoneinfo

pd.set_option('display.max_columns', 80)
pd.set_option('display.max_rows', 200)

People_df=pd.read_csv("../data/raw/Traffic_Crashes_-_People.csv.gz", low_memory=False, compression='gzip')
Crashes_df=pd.read_csv("../data/raw/Traffic_Crashes_-_Crashes.csv.gz",low_memory=False, compression='gzip')
Vehicle_df=pd.read_csv("../data/raw/Traffic_Crashes_-_Vehicles.csv.gz",low_memory=False, compression='gzip')


In [212]:
Crashes=Crashes_df.copy()
People=People_df.copy()
Vehicle=Vehicle_df.copy()

# INITIAL FILTERING BEFORE the JOIN, FOR PERFORMANCE REASONS:
Crashes=Crashes[Crashes['REPORT_TYPE']=='ON SCENE']
Crashes=Crashes.query("CRASH_TYPE == 'INJURY AND / OR TOW DUE TO CRASH'")

veh_type=['PASSENGER','SPORT UTILITY VEHICLE (SUV)','VAN/MINI-VAN','PICKUP','TRUCK - SINGLE UNIT','BUS OVER 15 PASS.']
Vehicle=Vehicle[Vehicle['VEHICLE_TYPE'].isin(veh_type)]
Vehicle.dropna(subset=['FIRST_CONTACT_POINT'], inplace=True)
Vehicle['VEHICLE_YEAR']= np.where(Vehicle['VEHICLE_YEAR'] < 1900, np.nan, Vehicle['VEHICLE_YEAR'])
Vehicle['VEHICLE_YEAR']= np.where(Vehicle['VEHICLE_YEAR'] > date.today().year, np.nan, Vehicle['VEHICLE_YEAR'])

People = People.query('PERSON_TYPE == "DRIVER"')
People = People.query('(AGE > 14 and AGE < 101) | AGE.isnull()', engine='python')  #
People = People.query('DRIVERS_LICENSE_CLASS in ("A","B","C","D","DM","AM","BM","CD") | DRIVERS_LICENSE_CLASS.isnull()', engine='python')
People=People.query('SEX!="X"')

temp_df1=pd.merge(People, Vehicle, on=['CRASH_RECORD_ID','VEHICLE_ID'])
ALL_df=pd.merge(temp_df1, Crashes, on='CRASH_RECORD_ID')

Index(['CRASH_UNIT_ID', 'CRASH_RECORD_ID', 'RD_NO', 'CRASH_DATE', 'UNIT_NO',
       'UNIT_TYPE', 'NUM_PASSENGERS', 'VEHICLE_ID', 'CMRC_VEH_I', 'MAKE',
       'MODEL', 'LIC_PLATE_STATE', 'VEHICLE_YEAR', 'VEHICLE_DEFECT',
       'VEHICLE_TYPE', 'VEHICLE_USE', 'TRAVEL_DIRECTION', 'MANEUVER',
       'TOWED_I', 'FIRE_I', 'OCCUPANT_CNT', 'EXCEED_SPEED_LIMIT_I', 'TOWED_BY',
       'TOWED_TO', 'AREA_00_I', 'AREA_01_I', 'AREA_02_I', 'AREA_03_I',
       'AREA_04_I', 'AREA_05_I', 'AREA_06_I', 'AREA_07_I', 'AREA_08_I',
       'AREA_09_I', 'AREA_10_I', 'AREA_11_I', 'AREA_12_I', 'AREA_99_I',
       'FIRST_CONTACT_POINT', 'CMV_ID', 'USDOT_NO', 'CCMC_NO', 'ILCC_NO',
       'COMMERCIAL_SRC', 'GVWR', 'CARRIER_NAME', 'CARRIER_STATE',
       'CARRIER_CITY', 'HAZMAT_PLACARDS_I', 'HAZMAT_NAME', 'UN_NO',
       'HAZMAT_PRESENT_I', 'HAZMAT_REPORT_I', 'HAZMAT_REPORT_NO',
       'MCS_REPORT_I', 'MCS_REPORT_NO', 'HAZMAT_VIO_CAUSE_CRASH_I',
       'MCS_VIO_CAUSE_CRASH_I', 'IDOT_PERMIT_NO', 'WIDE_LOAD_I',
       '

In [29]:
# GENERATING TARGET THROUGH binning and combining sec and prim causes:

targ_map ={ 'IMPROPER OVERTAKING/PASSING':'YES', 'UNABLE TO DETERMINE':'UNKNOWN',
       'IMPROPER BACKING':'YES', 'IMPROPER LANE USAGE':'YES',
       'UNDER THE INFLUENCE OF ALCOHOL/DRUGS (USE WHEN ARREST IS EFFECTED)':'YES',
       'DISREGARDING TRAFFIC SIGNALS':'YES',
       'FAILING TO REDUCE SPEED TO AVOID CRASH':'YES',
       'OPERATING VEHICLE IN ERRATIC, RECKLESS, CARELESS, NEGLIGENT OR AGGRESSIVE MANNER':'YES',
       'FAILING TO YIELD RIGHT-OF-WAY':'YES', 'EQUIPMENT - VEHICLE CONDITION':'NO',
       'VISION OBSCURED (SIGNS, TREE LIMBS, BUILDINGS, ETC.)':'NO',
       'IMPROPER TURNING/NO SIGNAL':'YES', 'FOLLOWING TOO CLOSELY':'YES',
       'DRIVING SKILLS/KNOWLEDGE/EXPERIENCE':'YES', 'DISREGARDING STOP SIGN':'YES',
       'NOT APPLICABLE':'UNKNOWN', 'DISTRACTION - FROM INSIDE VEHICLE':'NO',
       'DISTRACTION - FROM OUTSIDE VEHICLE':'NO',
       'HAD BEEN DRINKING (USE WHEN ARREST IS NOT MADE)':'YES',
       'ROAD ENGINEERING/SURFACE/MARKING DEFECTS':'NO',
       'DISREGARDING OTHER TRAFFIC SIGNS':'YES', 'TEXTING':'YES',
       'DRIVING ON WRONG SIDE/WRONG WAY':'YES', 'PHYSICAL CONDITION OF DRIVER':'YES',
       'ANIMAL':'NO', 'WEATHER':'NO', 'ROAD CONSTRUCTION/MAINTENANCE':'NO',
       'DISREGARDING YIELD SIGN':'YES', 'CELL PHONE USE OTHER THAN TEXTING':'YES',
       'EVASIVE ACTION DUE TO ANIMAL, OBJECT, NONMOTORIST':'NO',
       'TURNING RIGHT ON RED':'YES', 'RELATED TO BUS STOP':'NO',
       'DISTRACTION - OTHER ELECTRONIC DEVICE (NAVIGATION DEVICE, DVD PLAYER, ETC.)':'YES',
       'DISREGARDING ROAD MARKINGS':'YES', 'OBSTRUCTED CROSSWALKS':'NO',
       'PASSING STOPPED SCHOOL BUS':'YES',
       'EXCEEDING SAFE SPEED FOR CONDITIONS':'YES',
       'EXCEEDING AUTHORIZED SPEED LIMIT':'YES',
       'MOTORCYCLE ADVANCING LEGALLY ON RED LIGHT':'NO',
       'BICYCLE ADVANCING LEGALLY ON RED LIGHT':'NO'}

def create_target(this_driver_action, driver_error, driver_sec_error, phys_condition):
    if this_driver_action not in ['NONE','OTHER','UNKNOWN']:
        guilty='YES'
    elif this_driver_action == 'NONE':
        guilty='NO'
    elif this_driver_action in ['OTHER','UNKNOWN'] and phys_condition in ['MEDICATED','FATIGUED/ASLEEP','ILLNESS/FAINTED'] :
        guilty='NO'        
    else: 
        guilty = driver_error

    if guilty == 'UNKNOWN' and driver_error == 'UNKNOWN' and driver_sec_error == 'NO':
        guilty='NO'
    return guilty

ALL_df['SOME_DRIVER_ERROR']=ALL_df['PRIM_CONTRIBUTORY_CAUSE'].replace(targ_map)
ALL_df['SOME_DRIVER_SEC_ERROR']=ALL_df['SEC_CONTRIBUTORY_CAUSE'].replace(targ_map)

ALL_df['GUILTY']=ALL_df.apply(lambda row: create_target(row['DRIVER_ACTION'], row['SOME_DRIVER_ERROR'], 
                                                        row['SOME_DRIVER_SEC_ERROR'],row['PHYSICAL_CONDITION']), axis=1)

ALL_df = ALL_df[ALL_df['GUILTY']!='UNKNOWN']

In [30]:
ALL_df.shape

(143909, 151)

In [31]:
# Dividing on training and testing sets

y=ALL_df['GUILTY']
ALL_COL=ALL_df.drop(['GUILTY'],axis=1)
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(ALL_COL, y, random_state=100, stratify=y)

train_df=pd.concat([X_train,Y_train], axis=1)
test_df=pd.concat([X_test,Y_test], axis=1)

In [101]:
df=train_df.copy()

In [124]:
def age_categories (age, age_groups=[0, 16, 21, 24, 35, 55, 70, 80, 100]):
    bins=[x for x in age_groups]
    age_group=pd.cut(x=age, bins=bins)
    return age_group

def car_age_categories (year, age_groups=[0,3,8,15,25]):    
    latest=date.today().year
    oldest=1900
    
    bins=[x for x in age_groups]
    bins=np.sort(np.abs(np.array(bins)-latest))
    bins=np.insert(bins, 0, oldest)

    car_age_cat=pd.cut(x=[year], bins=bins)
    return car_age_cat

def select_license_class(dl_code):
    dl = 'Other'
    if dl_code in ["A","B","C","AM","BM","CD","S"]:
        dl='Commercial'
    elif dl_code in ('DM','D'):
        dl='Regular'
    return dl

def explore_cat_vars(df, n):
    col_counts=pd.DataFrame(df.nunique(),).reset_index()
    col_counts.columns=(["Col_Name", "Count"])
    for col in col_counts[col_counts['Count'] < n]['Col_Name']:
        print('--------------------------------------')
        print(col)
        print('--------------------------------------')
        print(df[col].value_counts())
    print("=================================================================================================")
    print(col_counts[col_counts['Count'] >= n])

[(16, 21]]
Categories (8, interval[int64, right]): [(0, 16] < (16, 21] < (21, 24] < (24, 35] < (35, 55] < (55, 70] < (70, 80] < (80, 100]]

In [171]:
col=['GUILTY','DEVICE_CONDITION', 'FIRST_CRASH_TYPE', 'FIRST_CONTACT_POINT', 
           'TRAFFICWAY_TYPE','ROADWAY_SURFACE_COND','ROAD_DEFECT','POSTED_SPEED_LIMIT','VEHICLE_DEFECT',
            'ALIGNMENT', 'LIGHTING_CONDITION', 'WEATHER_CONDITION','SEX','DRIVERS_LICENSE_CLASS',  
            'PHYSICAL_CONDITION', 'MANEUVER','NUM_PASSENGERS',  'DRIVER_VISION']



14758

In [ ]:
#df['INTERSECTION_RELATED_I']=df['INTERSECTION_RELATED_I'].fillna('UNKNOWN')

#df['FIRST_CONTACT_POINT']=df['FIRST_CONTACT_POINT'].fillna('UNKNOWN')






In [200]:
#proper_speed=np.arange(0,80,5)
#df=df[df['POSTED_SPEED_LIMIT'].isin(proper_speed)].copy()
df['POSTED_SPEED_LIMIT'].isna().sum()

0